In [2]:
import torch
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pickle
from torch.utils.data import TensorDataset, DataLoader

# Define the ResidualBlock class (same as in training)
class ResidualBlock(torch.nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.block = torch.nn.Sequential(
            torch.nn.Linear(dim, dim),
            torch.nn.BatchNorm1d(dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(dim, dim),
            torch.nn.BatchNorm1d(dim)
        )
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        return self.relu(self.block(x) + x)

# Define the ResNetRegressor class (same as in training)
class ResNetRegressor(torch.nn.Module):
    def __init__(self, input_dim):
        super(ResNetRegressor, self).__init__()
        self.input_layer = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        self.res_blocks = torch.nn.Sequential(
            ResidualBlock(512),
            ResidualBlock(512),
            ResidualBlock(512)
        )
        self.output_layer = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(256, 1)
        )
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.res_blocks(x)
        return self.output_layer(x)

# Paths
EMBEDDINGS_DIR = r'D:\amazon ml\test_embeddings'
CSV_PATH = 'test/merged_test_with_image.csv'  # Replace with your test CSV path if different
OUTPUT_DIR = 'fast_ensemble_models'
OUTPUT_CSV = 'resnet_predictions.csv'

# Load preprocessing objects
with open(os.path.join(OUTPUT_DIR, 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)
with open(os.path.join(OUTPUT_DIR, 'unit_encoder.pkl'), 'rb') as f:
    encoder = pickle.load(f)

# Load and process embeddings
image_pooled_list = []
text_pooled_list = []
for batch_idx in range(428):
    batch_dir = os.path.join(EMBEDDINGS_DIR, f'batch_{batch_idx}')
    image_emb_path = os.path.join(batch_dir, 'image_embeddings.pt')
    text_emb_path = os.path.join(batch_dir, 'text_embeddings.pt')
    
    if os.path.exists(image_emb_path) and os.path.exists(text_emb_path):
        try:
            image_emb = torch.load(image_emb_path)
            text_emb = torch.load(text_emb_path)
            image_pooled = image_emb.mean(dim=1).numpy()
            text_pooled = text_emb.mean(dim=1).numpy()
            image_pooled_list.append(image_pooled)
            text_pooled_list.append(text_pooled)
            del image_emb, text_emb
        except Exception as e:
            print(f"Error loading batch {batch_idx}: {e}")
            continue

if not image_pooled_list or not text_pooled_list:
    raise ValueError("No valid embeddings loaded.")

image_pooled = np.concatenate(image_pooled_list, axis=0)
text_pooled = np.concatenate(text_pooled_list, axis=0)
print(f"✓ Image embeddings: {image_pooled.shape}")
print(f"✓ Text embeddings: {text_pooled.shape}")
del image_pooled_list, text_pooled_list

# Load CSV
df = pd.read_csv(CSV_PATH)
print(f"✓ Loaded CSV: {len(df)} rows")

# Align data
num_samples = min(len(df), image_pooled.shape[0], text_pooled.shape[0])
df = df.iloc[:num_samples].reset_index(drop=True)
image_pooled = image_pooled[:num_samples]
text_pooled = text_pooled[:num_samples]

# Create features
embeddings_concat = np.concatenate([image_pooled, text_pooled], axis=1)
unit_encoded = encoder.transform(df[['unit']])
features = np.concatenate([embeddings_concat, df[['value']].to_numpy(), unit_encoded], axis=1)
print(f"Feature matrix: {features.shape}")

# Scale features
features_scaled = scaler.transform(features)

# Convert to tensor
features_tensor = torch.tensor(features_scaled, dtype=torch.float32)

# Load the ResNet model
input_dim = features.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetRegressor(input_dim).to(device)
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, 'resnet_model.pth')))
model.eval()

# Make predictions
with torch.no_grad():
    features_tensor = features_tensor.to(device)
    predictions = model(features_tensor).cpu().numpy().flatten()

# Create output DataFrame
output_df = pd.DataFrame({
    'sample_id': df['sample_id'],
    'price': predictions
})

# Save to CSV
output_df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✓ Predictions saved to '{OUTPUT_CSV}'")

# Print sample output
print("\nSample predictions:")
print(output_df.head().to_string(index=False))

c:\Users\harsh\miniconda3\envs\battery\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.7.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\harsh\miniconda3\envs\battery\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.7.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


✓ Image embeddings: (42800, 768)
✓ Text embeddings: (42800, 384)
✓ Loaded CSV: 57200 rows
Feature matrix: (42800, 1188)

✓ Predictions saved to 'resnet_predictions.csv'

Sample predictions:
 sample_id     price
    100179 18.377113
    245611 17.560154
    146263 23.193251
     95658  6.979241
     36806 21.656097
